In [1]:
from coffea.nanoevents import NanoEventsFactory, BaseSchema, PFNanoAODSchema
import json
import fastjet
import numpy as np
import awkward as ak
from coffea import processor
import hist
import coffea.nanoevents.methods.vector as vector
import warnings
import matplotlib.pyplot as plt
import math
from variable_functions import *
import os

/tmp/ipykernel_3153559/4208829920.py:8: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  import coffea.nanoevents.methods.vector as vector


In [2]:
warnings.filterwarnings("ignore", module="coffea.*")
warnings.filterwarnings("ignore", "Found duplicate branch")
warnings.filterwarnings("ignore", "Missing cross-reference index for")
warnings.filterwarnings("ignore", "dcut")
warnings.filterwarnings("ignore", "Please ensure")
warnings.filterwarnings("ignore", "invalid value")

In [40]:
files = os.listdir('/project01/ndcms/cmoore24/samples/hbb/')

In [41]:
events = NanoEventsFactory.from_root(
    [{'/project01/ndcms/cmoore24/samples/hbb/' + i: "/Events"} for i in files],
    delayed=True,
    schemaclass=PFNanoAODSchema,
    metadata={"dataset": "Hbb"},
).events()

In [42]:
cut_to_fix_softdrop = (ak.num(events.FatJet.constituents.pf, axis=2) > 0)
events = events[ak.all(cut_to_fix_softdrop, axis=1)]

In [13]:
fatjetSelect = (
            (events.FatJet.pt > 400)
            #& (events.FatJet.num_subjets >= 3)
            & (abs(events.FatJet.eta) < 2.4)
            & (events.FatJet.msoftdrop > 40)
            & (events.FatJet.msoftdrop < 200)
        )
# events = uncut_events[
#     ak.any(fatjetSelect, axis=1)
# ]

events["goodjets"] = events.FatJet[fatjetSelect]

# cut = (ak.num(uncut_events.FatJet.constituents.pf, axis=2) > 0)
# events = uncut_events[ak.all(cut, axis=1)]

In [15]:
num_goodjets_events = events.goodjets.pt.compute()

In [18]:
len(ak.flatten(num_goodjets_events))

37216

In [43]:
num_sub = ak.unflatten(num_subjets(events.FatJet, cluster_val=0.4), counts=ak.num(events.FatJet))
events['FatJet', 'num_subjets'] = num_sub
fatjetSelect = (
            (events.FatJet.pt > 400)
            & (events.FatJet.num_subjets >= 3)
            & (abs(events.FatJet.eta) < 2.4)
            & (events.FatJet.msoftdrop > 40)
            & (events.FatJet.msoftdrop < 200)
        )

events["goodjets"] = events.FatJet[fatjetSelect]

In [44]:
subcut_goodjets_events = events.goodjets.pt.compute()

In [46]:
len(ak.flatten(subcut_goodjets_events))

33945

In [23]:
(37216 - 33945)

3271

In [49]:
3271/37216

0.08789230438521066

In [45]:
len(subcut_goodjets_events)

497000

In [48]:
1-(497000-37216)/497000

0.07488128772635816

In [50]:
1-(497000-33945)/497000

0.0682997987927565

In [61]:
(0.0682997987927565)/(0.07488128772635816)

0.9121076956147887

In [26]:
files = os.listdir('/project01/ndcms/cmoore24/samples/hgg/')

In [27]:
events = NanoEventsFactory.from_root(
    [{'/project01/ndcms/cmoore24/samples/hgg/' + i: "/Events"} for i in files],
    delayed=True,
    schemaclass=PFNanoAODSchema,
    metadata={"dataset": "Hgg"},
).events()

In [28]:
cut_to_fix_softdrop = (ak.num(events.FatJet.constituents.pf, axis=2) > 0)
events = events[ak.all(cut_to_fix_softdrop, axis=1)]

In [29]:
# num_sub = ak.unflatten(num_subjets(events.FatJet, cluster_val=0.4), counts=ak.num(events.FatJet))
# events['FatJet', 'num_subjets'] = num_sub
fatjetSelect = (
            (events.FatJet.pt > 400)
            #& (events.FatJet.num_subjets >= 3)
            & (abs(events.FatJet.eta) < 2.4)
            & (events.FatJet.msoftdrop > 40)
            & (events.FatJet.msoftdrop < 200)
        )

events["goodjets"] = events.FatJet[fatjetSelect]

In [30]:
num_goodjets_events = events.goodjets.pt.compute()

In [31]:
len(ak.flatten(num_goodjets_events))

211906

In [32]:
num_sub = ak.unflatten(num_subjets(events.FatJet, cluster_val=0.4), counts=ak.num(events.FatJet))
events['FatJet', 'num_subjets'] = num_sub
fatjetSelect = (
            (events.FatJet.pt > 400)
            #& (events.FatJet.pt < 800)
            & (events.FatJet.num_subjets >= 3)
            & (abs(events.FatJet.eta) < 2.4)
            & (events.FatJet.msoftdrop > 40)
            & (events.FatJet.msoftdrop < 200)
        )

events["goodjets"] = events.FatJet[fatjetSelect]

In [33]:
subcut_goodjets_events = events.goodjets.pt.compute()

In [34]:
len(ak.flatten(subcut_goodjets_events))

199657

In [36]:
(211906-199657)/211906

0.057803931932083094

In [37]:
len(subcut_goodjets_events)

2392373

In [51]:
1-(2392373-211906)/2392373

0.0885756527096736

In [52]:
1-(2392373-199657)/2392373

0.0834556317096038

In [25]:
ak.where(ak.flatten(events.goodjets.pt) > 800)

(<Array [] type='0 * int64'>,)

In [148]:
events['FatJet', 'd2b1'] = ak.unflatten(
     d2_calc(events.FatJet), counts=ak.num(events.FatJet)
)

In [25]:
#events.FatJet.fields

In [28]:
import pandas as pd
a = pd.read_parquet('/project01/ndcms/cmoore24/skims/fatjet_skims/hgg', engine='fastparquet')

In [24]:
#a.keys()

In [62]:
files = os.listdir('/project01/ndcms/cmoore24/samples/qcd/800to1000')

In [66]:
events = NanoEventsFactory.from_root(
    {'/project01/ndcms/cmoore24/samples/qcd/800to1000/' + files[0]: "/Events"},
    delayed=True,
    schemaclass=PFNanoAODSchema,
    metadata={"dataset": "QCD_800to1000"},
).events()

In [67]:
cut_to_fix_softdrop = (ak.num(events.FatJet.constituents.pf, axis=2) > 0)
events = events[ak.all(cut_to_fix_softdrop, axis=1)]

In [68]:
# num_sub = ak.unflatten(num_subjets(events.FatJet, cluster_val=0.4), counts=ak.num(events.FatJet))
# events['FatJet', 'num_subjets'] = num_sub
fatjetSelect = (
            (events.FatJet.pt > 400)
            #& (events.FatJet.num_subjets >= 3)
            & (abs(events.FatJet.eta) < 2.4)
            & (events.FatJet.msoftdrop > 40)
            & (events.FatJet.msoftdrop < 200)
        )

events["goodjets"] = events.FatJet[fatjetSelect]

In [69]:
num_goodjets_events = events.goodjets.pt.compute()

In [70]:
len(ak.flatten(num_goodjets_events))

11363

In [71]:
num_sub = ak.unflatten(num_subjets(events.FatJet, cluster_val=0.4), counts=ak.num(events.FatJet))
events['FatJet', 'num_subjets'] = num_sub
fatjetSelect = (
            (events.FatJet.pt > 400)
            #& (events.FatJet.pt < 800)
            & (events.FatJet.num_subjets >= 3)
            & (abs(events.FatJet.eta) < 2.4)
            & (events.FatJet.msoftdrop > 40)
            & (events.FatJet.msoftdrop < 200)
        )

events["goodjets"] = events.FatJet[fatjetSelect]

In [72]:
subcut_goodjets_events = events.goodjets.pt.compute()

In [73]:
len(ak.flatten(subcut_goodjets_events))

10369

In [74]:
len(subcut_goodjets_events)

19000

In [75]:
1-(19000-11363)/19000

0.5980526315789474

In [76]:
1-(19000-10369)/19000

0.5457368421052631

In [77]:
0.5457368421052631/0.5980526315789474

0.9125231012936723

In [78]:
files = os.listdir('/project01/ndcms/cmoore24/samples/wqq/800toInf')

In [79]:
events = NanoEventsFactory.from_root(
    {'/project01/ndcms/cmoore24/samples/wqq/800toInf/' + files[0]: "/Events"},
    delayed=True,
    schemaclass=PFNanoAODSchema,
    metadata={"dataset": "Wqq_800toInf"},
).events()

In [80]:
cut_to_fix_softdrop = (ak.num(events.FatJet.constituents.pf, axis=2) > 0)
events = events[ak.all(cut_to_fix_softdrop, axis=1)]

In [81]:
# num_sub = ak.unflatten(num_subjets(events.FatJet, cluster_val=0.4), counts=ak.num(events.FatJet))
# events['FatJet', 'num_subjets'] = num_sub
fatjetSelect = (
            (events.FatJet.pt > 400)
            #& (events.FatJet.num_subjets >= 3)
            & (abs(events.FatJet.eta) < 2.4)
            & (events.FatJet.msoftdrop > 40)
            & (events.FatJet.msoftdrop < 200)
        )

events["goodjets"] = events.FatJet[fatjetSelect]

In [82]:
num_goodjets_events = events.goodjets.pt.compute()

In [83]:
len(ak.flatten(num_goodjets_events))

14697

In [84]:
num_sub = ak.unflatten(num_subjets(events.FatJet, cluster_val=0.4), counts=ak.num(events.FatJet))
events['FatJet', 'num_subjets'] = num_sub
fatjetSelect = (
            (events.FatJet.pt > 400)
            #& (events.FatJet.pt < 800)
            & (events.FatJet.num_subjets >= 3)
            & (abs(events.FatJet.eta) < 2.4)
            & (events.FatJet.msoftdrop > 40)
            & (events.FatJet.msoftdrop < 200)
        )

events["goodjets"] = events.FatJet[fatjetSelect]

In [85]:
subcut_goodjets_events = events.goodjets.pt.compute()

In [86]:
len(ak.flatten(subcut_goodjets_events))

13238

In [87]:
len(subcut_goodjets_events)

27083

In [88]:
1-(27083-14697)/27083

0.5426651404940368

In [89]:
1-(27083-13238)/27083

0.48879370823025514

In [90]:
0.48879370823025514/0.5426651404940368

0.9007280397360006